In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import collections

# Start with pct% of population infected
def init_graph(initial_infection = .05, graph_model = 'relaxed_caveman',
               pop_size = 1000, seed = None):
    if graph_model == 'relaxed_caveman':
        G = nx.relaxed_caveman_graph(int(pop_size/4), 4, 0.25, seed)
    elif graph_model == 'scale_free':
        G = nx.scale_free_graph(pop_size, seed=seed)
    else:
        raise ValueError("Unknown graph type")
        
    init_infection(G, initial_infection)

    return G

def init_parameters(initial_infection, graph_model, pop_size = 1000, seed=None):
    G = init_graph(initial_infection, graph_model, pop_size, seed)
    
    status = current_status(G)
    
    pop = len(G.nodes)
    i = status['infected'] / pop
    s = (pop - i) / pop
    newly_infected = status['infected']
    r, contacts_infected = 0, 0

    data = [[s,i, r, newly_infected, contacts_infected]]

    return G, data, status, pop

def init_infection(G, pct):
    """
    Given a Graph G, infects pct% of population and set the remainder as susceptible.
    This is considered day 0.
    """
    size = int(len(G.nodes) * pct) 
    infected = np.random.choice(G.nodes, size = size)
    
    for node in G.nodes():
        if node in infected: 
            G.nodes[node].update({'status' : 'infected',
                                  'infection_day': 0, 
                                  'contacts_infected': 0})
        else:
            G.nodes[node].update({'status': 'susceptible', 
                                  'infection_day' : -1, 
                                  'contacts_infected': 0})



def recover_one_step(G, day, recover_time = 12):
    """
    Recover everyone that has been infected recover_time days or more
    """

    for node, adjacencies in enumerate(G.adjacency()):
        if G.nodes[node]['status'] == 'infected':
            if np.random.random() < 1/15:
                G.nodes[node]['status'] = 'recovered'
            #if day - G.nodes[node]['infection_day'] >= recover_time: 
            #    G.nodes[node]['status'] = 'recovered'

def spread_one_step(G, day, p_r = 0.5, lambda_leak = 0.05):
    """
    Spreads the infection 1 step, to the susceptible neighbours of infected people
    day is current day
    """
    newly_infected = []
       
    for node, adjacencies in enumerate(G.adjacency()):
        if G.nodes[node]['status'] == 'susceptible':
            if np.random.random() < lambda_leak:
                newly_infected.append(node)    
            else:
                for contact in adjacencies[1].keys():
                    if G.nodes[contact]['status'] == 'infected' and np.random.random() < p_r:
                            newly_infected.append(node)
                            G.nodes[contact]['contacts_infected'] += 1
                            break  
        
    for node in np.unique(newly_infected):
        G.nodes[node].update({'status' : 'infected', 'infection_day': day})
        
    return len(newly_infected)


def simulate_one_step(G, day, recover_time=12, p_r=0.5, infectious_window=[4,6]):
    """
    Recover and Spread one step
    """
    recover_one_step(G, day, recover_time)
    newly_infected =  spread_one_step(G, day, p_r, infectious_window)
    return newly_infected

def current_status(G):
    """
    Returns a dict containing the current status of susceptible, infected and recovered
    """
    nodes = np.array(G.nodes(data=True))[:,1]
    result = collections.Counter(node['status'] for node in nodes)
    return result

def get_mean_contacts_infected(G):
        contacts_infected = [node['contacts_infected'] for i, node in G.nodes(data=True)\
                                                             if node['status'] == 'recovered']
        if len(contacts_infected) > 0: 
            contacts_infected = np.mean(contacts_infected)
        else:
            contacts_infected = np.nan
            
        return contacts_infected
    
def get_time_series_row(G, pop):
    status = current_status(G)
    s = status['susceptible'] / pop
    i = status['infected'] / pop
    r = status['recovered'] / pop

    contacts_infected = get_mean_contacts_infected(G)
    
    return s, i, r, contacts_infected, status
    
def simulate_pandemic(initial_infection=.05, recover_time=12, p_r=.5, lambda_leak=.05,
                      graph_model = 'relaxed_caveman', pop_size = 1000,
                      seed = None):
    """
    Runs the course of the pandemic from the start until
    less than 1% of the population is simultaneously infected or no one is infected
    """
    
    np.random.seed(seed)
    
    G, data, status, pop = init_parameters(initial_infection, graph_model, pop_size, seed)

    for day in range(150):
        
        if (status['recovered']+status['susceptible'])>=pop:
            break
    
        recover_one_step(G, day, recover_time)
        
        newly_infected = spread_one_step(G, day, p_r, lambda_leak)
       
        s, i, r, contacts_infected, status = get_time_series_row(G, pop)

        data.append([s, i, r, newly_infected, contacts_infected])
        
    columns = ['susceptible', 'infected', 'recovered', 'newly_infected', 'contacts_infected_mean']

    time_series = pd.DataFrame(data, columns=columns)
    
    return time_series


In [6]:
import plotly.graph_objects as go

df_1 = simulate_pandemic(graph_model='scale_free')
df_2 = simulate_pandemic(graph_model='scale_free')

fig = go.Figure()
x1 = df_1.index.to_list()
x2 = df_2.index.to_list()

y1 = df_1['infected']
y2 = df_2['infected']

fig.add_trace(go.Scatter(x=x1, y=y1,
    fill=None,
    mode='lines',
    line_color='magenta',
     name="Sim 1"
    ))


fig.add_trace(go.Scatter(x=x2, y=y2,
    fill=None,
    mode='lines',
    line_color='indigo',
     name="Sim 2"
    ))

In [8]:
df_1 = simulate_pandemic(graph_model='scale_free', seed=13)
df_2 = simulate_pandemic(graph_model='scale_free', seed=13)

import plotly.graph_objects as go

fig = go.Figure()
x1 = df_1.index.to_list()
x2 = df_2.index.to_list()

y1 = df_1['infected']
y2 = df_2['infected']

fig.add_trace(go.Scatter(x=x1, y=y1,
    fill=None,
    mode='lines',
    line_color='magenta',
     name="Sim 1"
    ))


fig.add_trace(go.Scatter(x=x2, y=y2,
    fill=None,
    mode='lines',
    line_color='indigo',
     name="Sim 2"
    ))

In [4]:
import plotly.graph_objects as go

df_1 = simulate_pandemic(seed=13)
df_2 = simulate_pandemic(seed=13)

fig = go.Figure()
x1 = df_1.index.to_list()
x2 = df_2.index.to_list()

y1 = df_1['infected']
y2 = df_2['infected']

fig.add_trace(go.Scatter(x=x1, y=y1,
    fill=None,
    mode='lines',
    line_color='magenta',
     name="Sim 1"
    ))


fig.add_trace(go.Scatter(x=x2, y=y2,
    fill=None,
    mode='lines',
    line_color='indigo',
     name="Sim 2"
    ))